# Data Cleaning and Preparation Notebook

This notebook focuses on the initial stages of the data analytics pipeline. Here, we'll perform the following key tasks:
- Import necessary libraries and set initial configurations.
- Collect and validate user inputs regarding stock analysis preferences.
- Retrieve stock data and related metrics from the Alphavantage API.
- Perform preliminary data cleaning and validation.

The cleaned data will serve as the foundation for subsequent analysis and modeling in the later notebooks.

### Library Imports and Initial Setup

In this section, we import all the necessary libraries and set up initial configurations. This prepares our environment for data analysis and visualization.

In [1]:
import requests
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import scipy.optimize as sco
import warnings
import os
warnings.filterwarnings('ignore')

In [3]:
# Default stock tickers and index for analysis, these will be the default choice if user didn't provide any from their side.

test_list = ['AAPL', 'ORCL', 'NVDA']
index = ['SPY']

term = 'short'

In [4]:
# Loading a list of valid stock tickers from a file.

# Note: Within this project, we are currently only be able to accept stocks listed on the S&P 500.
with open('stock ticker', 'r') as file:
    valid_tickers = set(file.read().splitlines())

### User Input and Data Validation

Here, we prompt the user to select the desired time interval and stock tickers for analysis. We also validate the chosen stock tickers against a predefined list.

- It promt the user to choose between a 'long' or 'short' time interval for anlalyse.
- it will also want the user to choose what stocsk they want to analyse, with a max limit of 3 stocks.
- if not input were give, it will have a default value of short, and stock list of: 'AAPL', 'ORCL', 'NVDA'.

In [5]:
# Function to check if a given stock ticker is valid.

def is_valid_ticker(ticker):
    return ticker.upper() in valid_tickers

In [6]:
# Here, we want to know if the user want to use long term data, or a short term data.
user_input = input("Enter 'short' or 'long' for the time interval you want to analyse data for(or press Enter to use the default): ")

while user_input not in ['long', 'short', '']:
    print("Invalid input. Please enter either 'long', 'short', or press Enter to use the default.")
    user_input = input("Enter the term to work with ('long', 'short' or press Enter to use the default): ").lower()

# Check if the user pressed Enter without typing anything, this will mean the term will set to short by default:
if user_input == "":
    pass
else:
    # Otherwise, use the user's input
    term = user_input

print(f"You have selected: {term}")


Enter 'short' or 'long' for the time interval you want to analyse data for(or press Enter to use the default): 
You have selected: short


In [7]:
# Here, we attempt to collect stock tickers from the user.
# If the user doesn't specify any ticker, the default list is used.

list_copy = test_list.copy()
default_use = False

# Loop three times to collect three stock tickers
for i in range(1, 4):
    # Prompt the user for a stock ticker
    ticker = input(f"Enter the stock ticker for stock (or press Enter to use the default) {i}: ")
    
    if ticker == "":
        print("you have choosed the default stock list")
        default_use = True
        break
        
    # Validaing the user input using the pre define function:
    while not is_valid_ticker(ticker):
        print("Invalid ticker. Please try again.")
        ticker = input("Enter a stock ticker: ") 
        
    # Add the ticker to the list
    list_copy[i-1] = (ticker.upper())  # Assuming tickers should be uppercase

# Now stock_tickers contains the three tickers entered by the user
if default_use == True:
    list_copy = test_list.copy()
    
print(f"You have choosed the following stock tickers: {', '.join(list_copy)}")

Enter the stock ticker for stock (or press Enter to use the default) 1: 
you have choosed the default stock list
You have choosed the following stock tickers: AAPL, ORCL, NVDA


In [8]:
# Here, we ensure that we also get data for the market index.
list_copy = list_copy + index

### Data Retrieval from Alphavantage API

In this section, we define functions to fetch stock data, market data, and risk-free rate data from the Alphavantage API. This data is crucial for our subsequent analysis.

In [9]:
# Below are the functions to fetch stock and risk free rate data from Alphavantage API based on the chosen term.
# Data is fetched for each stock in the list.
# API rate limits are respected by introducing sleep between requests.

def get_price(term):
    
    # Note: The API from alphavintage will need the user to get their own.
    # Here, I have used my own which is saved in my local machine. 
    key_alpha = os.environ.get('KEY_ALPHA') 
    BASE_URL = 'https://www.alphavantage.co/query?'
    data = {}
    
    if term == 'short':
        function = 'TIME_SERIES_DAILY'
    else:
        function = 'TIME_SERIES_MONTHLY'
        
    for stock in list_copy:
        params = {
                  'function': function,
                  'symbol': stock,
                  'apikey': key_alpha
                 }

        response = requests.get(BASE_URL, params = params)

        if response.status_code == 200:
            data[stock] = response.json()  # get data succuse

            # Respect the rate limit from alpha vintage. (Note: this is what slowing down the run speed of the file)
            time.sleep(15)
        
        else:
            print(f"Failed to retrieve data for {stock}. Status code: {response.status_code}")
    
    return data

In [10]:
# Function for getting risk free rate data:

def get_rf(term):

    key_alpha = os.environ.get('KEY_ALPHA')
    BASE_URL = 'https://www.alphavantage.co/query?'
    data = {}
    
    if term == 'short':
        interval = 'daily'
    else:
        interval = 'monthly'
    
    params_yield = {
        'function': 'TREASURY_YIELD',
        'apikey': key_alpha,
        'interval': interval
                    }
    response = requests.get(BASE_URL, params = params_yield)

    if response.status_code == 200:
            data = response.json()  # get data succuse
        
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
    
    return data

In [11]:
def date_range_fix(term, data):
    if term == 'short':
        cutoff_date = '2023-07-31'
        data = data[data['date'] <= cutoff_date]
    
    else:
        cutoff_date1 = '2023-07-31'
        cutoff_date2 = '2013-07-31'
        data = data[data['date'] <= cutoff_date1 & data['date'] >= cutoff_date2]
        
    return data

In [12]:
# Transforming and formatting the fetched stock data into a structured DataFrame.
# Data for individual stocks is concatenated, and columns are standardized.

# Getting stock data:
stock_data = get_price(term)

# Getting risk free rate data:
rf_data = get_rf(term)

### Loading and Preparing Factor Data

We load factors data required for our models from CSV files. This includes data for both the Fama model and the Momentum model.

In [13]:
# Getting factors data for fama model:
new_factor = pd.read_csv('F-F_Research_Data_Factors_daily.CSV', delimiter = ',', parse_dates = ['date'])

In [14]:
# Getting factor data for momentum model:
mom_factor = pd.read_csv('F-F_Momentum_Factor_daily.CSV', delimiter = ',', parse_dates = ['date'])

# Here, to prevent the parse_dates function with pd from not working sometime with different date format, we need to convert it ourself
mom_factor['date'] = pd.to_datetime(mom_factor['date'], format='%Y%m')

### Data Formatting and Transformation

The raw data retrieved and loaded in previous steps is transformed into structured DataFrames. We ensure that the data is in the correct format and structure for analysis.

In [15]:
# Performing data cleaning tasks, renaming columns, and adjusting the date range based on the chosen term.

frame_list = []

for ticker, datas in stock_data.items():
    df = pd.DataFrame(datas['Time Series (Daily)']).T
    df['ticker'] = ticker  # add ticker column for the each stock
    df = df.reset_index().rename(columns = {'index': 'date'})  # Reset the index and rename it to "date"
    df['date'] = pd.to_datetime(df['date'])  # Convert the date column to datetime format
    frame_list.append(df)  # Append the DataFrame to the list


# After extracting needed data from the pulled data, we can now build them into one dataframe:
combine = pd.concat(frame_list[0:3]) # we only use the first 3, because the last one is the index

market_return = frame_list[-1] # this is the index, we will deal with it more in later part.


# Give new columns name:
column_name = ['date', 'open', 'high', 'low', 'close', 'volumn', 'ticker']
combine.columns = column_name
market_return.columns = column_name


# Changing types so that we can make calculation with our datas:
combine.iloc[:, 2:6] = combine.iloc[:, 2:6].astype(float)
market_return.iloc[:, 2:6] = market_return.iloc[:, 2:6].astype(float)

In [16]:
# date type conversion for risk free rate dataframe as well:
rf_data = pd.DataFrame(rf_data['data'])
rf_data['date'] = pd.to_datetime(rf_data['date'])

In [17]:
# This is the dataframe use for latter part of this project:

backtesting_data = combine[(combine['date'].dt.year == 2023) & ((combine['date'].dt.month == 8) | (combine['date'].dt.month == 9))]
backtesting_data = backtesting_data.iloc[::-1].reset_index(drop=True)

In [18]:
# merge the stock data and index data:
merged_data = combine.merge(rf_data, on = 'date', how = 'left').merge(market_return[['date','close']], on='date', how='left')

In [19]:
# fix up only small detail:

merged_data['value'] = merged_data['value'].astype(float) / 100 # have the risk free rate in numeric form, rather than percent form.

# Renaming some of the columns:
merged_data = merged_data.rename(columns = {'value': 'rf'})
merged_data = merged_data.rename(columns = {'close_y': 'market'})

# Set the date range for our dataframe using the date_range_fix( function we defined above:
merged_data = date_range_fix(term, merged_data)
merged_data = merged_data.iloc[::-1].reset_index(drop=True)

### Computing Returns and Excess Returns

In this section, we compute daily returns for the selected stocks and the market. We also calculate the excess returns by considering the risk-free rate.

In [20]:
# getting the daliy return:
merged_data["ret"] = merged_data.groupby('ticker')['close_x'].apply(lambda x: x.pct_change(1))
merged_data["mkret"] = merged_data.groupby('ticker')['market'].apply(lambda x: x.pct_change(1))

In [21]:
# Getting excess returns:
merged_data['ex_return'] = merged_data['ret'] - merged_data['rf']
merged_data['mkt_ex_return'] = merged_data['mkret'] - merged_data['rf']

### Merging All Relevant Data

Finally, we merge all the relevant data sources (stock data, risk-free rate, market returns, and factors data) into a single comprehensive dataset. This dataset forms the basis for any subsequent analysis and modeling.

In [22]:
# merging all factors data needed:
final_data = merged_data.merge(new_factor[['date', 'SMB', 'HML']], on = 'date', how = 'left').merge(mom_factor[['date', 'Mom']], on = 'date', how = 'left')

In [5]:
final_data

NameError: name 'final_data' is not defined